### **Word2Vec**

원 핫 인코딩을 사용하면서도 단어 간 유사도를 반영할 수 있도록 단어의 의미를 벡터화하는 방법이다.

*비슷한 위치에서 등장하는 단어들은 비슷한 의미를 가진다*는 분포 가설을 따르는 분산 표현 방법을 사용한다.

예) '강아지'는 주로 '귀엽다','예쁘다' 등의 단어와 함께 등장하는데, 이러한 내용을 가지는 텍스트를 벡터화하면 이 단어들은 의미적으로 가까운 단어가 된다.

<br>
Word2Vec에는 다음 두 가지 방식이 있다.  

- CBOW(Continous Bag of Words)
- Skip-Gram


#### **CBOW(Continous Bag of Words)**

주변에 있는 단어들을 통해 중간에 있는 단어를 예측하는 방법이다.  
윈도우를 두고, 윈도우 내의 주변 단어의 벡터로 중심 단어의 벡터를 예측한다.    
Skip-Gram에 비해 몇 배 빠른 훈련이 가능하며, 빈번한 단어를 예측하는 데 더 나은 정확도를 가진다.

In [1]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [28]:
import os
import re
from pathlib import Path

In [31]:
BASE_DIR = "/data/ksb/TestDir"
DATA_BASE_DIR = os.path.join(BASE_DIR, "sample_articles")

ORIGIN_PATH = os.path.join(DATA_BASE_DIR,"Origin-Data")
PREPROCESSED_PATH = os.path.join(DATA_BASE_DIR,"Preprocessed-Data")
PRETTY_PATH = os.path.join(DATA_BASE_DIR,"Pretty-Data")
SWORDS_PATH = os.path.join(DATA_BASE_DIR, "StopWordList.txt")
MODEL_PATH = os.path.join(os.path.join(Path(os.getcwd()).parent, "Word-Embedding-Model"))

In [24]:
def mkdir_p(path):
    import errno
    try:
        os.makedirs(path)
    except OSError as exc:
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else:
            raise

def del_folder(path):
    from shutil import rmtree
    try:
        rmtree(path)
    except Exception as e:
        print(e)
        pass

In [8]:
class RawTextReader:
    def __init__(self, filepath):
        self.filepath = filepath
        self.rgxSplitter = re.compile("/n")

    def __iter__(self):
        for line in open(self.filepath, encoding='utf-8'):
            ch = self.rgxSplitter.split(line)
            for s in ch:
                yield s

경향신문 언론사 첫번째 기사를 예시로 전처리된 텍스트를 토큰화 한다.

In [9]:
media_list = os.listdir(ORIGIN_PATH)

media = media_list[2]
media_path = os.path.join(PREPROCESSED_PATH, media)
article_list= os.listdir(media_path)

article = article_list[0]
reader = RawTextReader(os.path.join(media_path, article))
    
content = list(filter(None, reader))
content

['진행',
 '강진원 앵커 박상연 앵커 출연',
 '김광삼 변호사 재원 부산 가톨릭대 특 임 교수 아래 텍스트 실제 방송 내용 차이 있 있 내용 방송 확인 바라',
 '대법원 선고 결과 이재명 경기 지사 관련 대법원 선고 결과 관련 전문가 내용 짚',
 '변호사 주문 있 제가 읽',
 '다수 의견 다음 판결',
 '원심 판결 유무 죄 부분 포함 유죄 부분 파기 부분 사건 수원 고등법원 환송',
 '허위 사실 공표 관련 내용 무죄 취지 파기 환송 이렇 보',
 '말씀',
 '무죄 취지 파기 환송 하 이건 서울고등법원 재판 하 되',
 '대법원 무죄 취지 서울 고등 취지 귀속 되 없',
 '결과 이재명 지사 원고 무죄 판결 받 되 이렇 보 있',
 '대하 만약 검찰 상고 하 되 대법원 재판 받 되 그렇 결과 영향 미치 선고 나오 이렇 보 있',
 '실익 없 검찰 입장',
 '맞',
 '사실 일각 사건 전원 합의체 넘어가 자체 지사 긍정 있 시각 있',
 '맞',
 '전원 합의체 넘어가 이례 중계 대법원 결정',
 '그렇 많 전문가 심의 결과 반대 결과 차원 대법원 부분 홍보 하 생각 들 오늘 전원 합의체 다수 의견 자체 허위 사실 공표 죄 새롭 판례 확립',
 '측면 부분 적극 알리 하 부분 반영 차원 이재명 지사 대하 무죄 선고 자체 강도 높 예측 순간',
 '이재명 경기 지사 같 경우 여권 차기 대선 주자 꼽히 인물 오늘 선고 결과 대하 관심 정치 뜨겁',
 '정치 반응 나오 같 국회 연결 알아보',
 '오늘 판결 대하 정치 반응 정리',
 '더불어민주당 한숨 돌리 분위기',
 '지사 여권 대권 가도 이낙연 의원 뒤쫓 경쟁자',
 '지사 원심 확정 판결 받 내후년 대통령 선거 출마 없 되 민주당 대선 후보 잃 되 위기',
 '대선 주자 안희정 충남 지사 박원순 서울 시장 낙 마한 상황 차기 대권 구도 적신호 켜 우려 나오',
 '무죄 취지 파기 환송 결정 나 걱정 씻 되',
 '지사 정치 위기 벗어나 남은 동안 도정 안정 운영 동력 얻 되',
 '이낙연 의원

In [10]:
tokenList = [sent.split() for sent in content]
tokenList

[['진행'],
 ['강진원', '앵커', '박상연', '앵커', '출연'],
 ['김광삼',
  '변호사',
  '재원',
  '부산',
  '가톨릭대',
  '특',
  '임',
  '교수',
  '아래',
  '텍스트',
  '실제',
  '방송',
  '내용',
  '차이',
  '있',
  '있',
  '내용',
  '방송',
  '확인',
  '바라'],
 ['대법원',
  '선고',
  '결과',
  '이재명',
  '경기',
  '지사',
  '관련',
  '대법원',
  '선고',
  '결과',
  '관련',
  '전문가',
  '내용',
  '짚'],
 ['변호사', '주문', '있', '제가', '읽'],
 ['다수', '의견', '다음', '판결'],
 ['원심',
  '판결',
  '유무',
  '죄',
  '부분',
  '포함',
  '유죄',
  '부분',
  '파기',
  '부분',
  '사건',
  '수원',
  '고등법원',
  '환송'],
 ['허위', '사실', '공표', '관련', '내용', '무죄', '취지', '파기', '환송', '이렇', '보'],
 ['말씀'],
 ['무죄', '취지', '파기', '환송', '하', '이건', '서울고등법원', '재판', '하', '되'],
 ['대법원', '무죄', '취지', '서울', '고등', '취지', '귀속', '되', '없'],
 ['결과', '이재명', '지사', '원고', '무죄', '판결', '받', '되', '이렇', '보', '있'],
 ['대하',
  '만약',
  '검찰',
  '상고',
  '하',
  '되',
  '대법원',
  '재판',
  '받',
  '되',
  '그렇',
  '결과',
  '영향',
  '미치',
  '선고',
  '나오',
  '이렇',
  '보',
  '있'],
 ['실익', '없', '검찰', '입장'],
 ['맞'],
 ['사실', '일각', '사건', '전원', '합의체', '넘어가', '자체', '지사', '긍정', '있

#### **Word2Vec 학습**

gensim에서는 다음과 같이 Word2Vec을 지원한다.  

> `sentences` : 단어 토큰화된 문장
> `size` : Projection Layer의 크기, 임베딩 벡터의 차원  
> `window` : 윈도우 크기  
> `min_count` : 단어의 최소 빈도 수, 이 이하의 빈도를 가지는 단어는 학습하지 않음  
> `workers` : 학습을 위한 프로세스 수  
> `sg` : 0일 경우 CBOW, 1일 경우 Skip-Gram  



In [11]:
from gensim.models import Word2Vec
model = Word2Vec(sentences=tokenList, size=100, window=5, min_count=1, workers=4, sg=0)

'선거'를 입력했을 때 의미가 유사한 단어들을 출력합니다.

In [12]:
model_result = model.wv.most_similar("선거")
model_result

[('이유', 0.3242764472961426),
 ('지나', 0.30343538522720337),
 ('교체', 0.30158963799476624),
 ('말씀', 0.2990836501121521),
 ('춘천', 0.29199838638305664),
 ('자신감', 0.2813935875892639),
 ('사실관계', 0.28133994340896606),
 ('있', 0.26314792037010193),
 ('대선', 0.25688642263412476),
 ('지급', 0.24449597299098969)]

#### **Skip-Gram**

중심 단어를 통해 주변에 있는 단어들을 예측하는 방법이다.  
소량의 학습 데이터에서도 잘 동작하며, 자주 사용하지 않는 희귀한 단어를 예측할 수 있다. 하지만 계산 비용이 크다는 문제점이 있다.   
마찬가지로 중심 단어에 윈도우를 두고, 윈도우 내의 주변 단어의 임베딩 벡터를 예측한다.

In [13]:
model = Word2Vec(sentences=tokenList, size=100, window=5, min_count=1, workers=4, sg=1)

In [14]:
model_result = model.wv.most_similar("선거")
model_result

[('있', 0.8363155126571655),
 ('하', 0.8078051805496216),
 ('지사', 0.7888187766075134),
 ('이번', 0.7623277306556702),
 ('대법원', 0.753893256187439),
 ('심', 0.7500892877578735),
 ('선고', 0.7465951442718506),
 ('사건', 0.7428316473960876),
 ('대선', 0.7383707761764526),
 ('입장', 0.7377241849899292)]

기사 본문 내용이 짧기 때문에, 모델을 학습하기에 corpus의 크기가 작다.  
유사도 또한 상당히 낮다.   
아래는 수집한 기사 100,000건들을 통해 corpus를 구성하고, Word2Vec 모델을 구축하는 내용이다.

In [15]:
media_list = os.listdir(PREPROCESSED_PATH)

result = []
forCount = []
for media in media_list:
    media_path = os.path.join(PREPROCESSED_PATH, media)
    article_list= os.listdir(media_path)

    for article in article_list:
        reader = RawTextReader(os.path.join(media_path, article)) 
        content = list(filter(None, reader))
        forCount += [token for sent in content for token in sent.split()]
        result += [sent.split() for sent in content]

In [16]:
print("전체 token의 개수 : {len}".format(len=len(forCount)))
print("중복되지 않은 token의 개수 : {len}".format(len=len(list(set(forCount)))))

전체 token의 개수 : 19644153
중복되지 않은 token의 개수 : 66456


In [17]:
START_TOKEN = ['<SOS>']
END_TOKEN = ['<EOS>']

In [18]:
for idx, i in enumerate(result):
    try:
        i =START_TOKEN + i +END_TOKEN
    except:
        print(idx, type(i), i)

In [19]:
result = list(map(lambda content : START_TOKEN + content + END_TOKEN, result))

In [20]:
model = Word2Vec(sentences=result, size=256, window=5, min_count=1, workers=4, sg=1, iter=10)

In [21]:
model_result = model.wv.most_similar("선거")
model_result

[('총선', 0.6850500106811523),
 ('선거일', 0.6091201901435852),
 ('낙선', 0.6031482815742493),
 ('대선', 0.5916657447814941),
 ('총선거', 0.5857176184654236),
 ('재보궐선거', 0.584223210811615),
 ('후보', 0.5775339603424072),
 ('투표일', 0.5758824348449707),
 ('금권', 0.5721417665481567),
 ('보궐', 0.5661484599113464)]

In [22]:
model.wv.__getitem__('선거').shape

(256,)

In [32]:
mkdir_p(MODEL_PATH)

wordvec_path = os.path.join(MODEL_PATH, "word2vec-256.wordvectors")
model_path = os.path.join(MODEL_PATH, 'word2vec-256.model')

word_vectors = model.wv
word_vectors.save(wordvec_path)
model.save(model_path)